!pip install -U spacy

!pip install Arabic-Stopwords

conda install -c conda-forge spacy-model-en_core_web_sm

!pip install nltk --quite

In [1]:
import pandas as pd
import re
import string
import sys
import argparse
import spacy
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mjeed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import arabicstopwords.arabicstopwords as stp

In [49]:
from nltk.corpus import stopwords

In [4]:
df = pd.read_csv(r"Arabic Poem Comprehensive Dataset (APCD).csv")

In [80]:
df.head(10)

,era,poet,poetical_works,rhyme,measure,r_hemistich,l_hemistich,line_of_poetry,line_of_poetry_split,line_of_poetry_split_stop
0,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأن تجمعا شملي وتنتظرا غدا,خليلي لا تستعجلا أن تزودا,خليلي تستعجلا ان تزودا وان تجمعا شملي وتنتظرا,"[خليلي, تستعجلا, ان, تزودا, وان, تجمعا, شملي, ...","[خليلي, تستعجلا, ان, تزودا, وان, تجمعا, شملي, ..."
1,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,ولا سرعتي يوما بسابقة الردى,فما لبث يوما بسابق مغنم,فما لبث يوما بسابق مغنم سرعتي يوما بسابقه الردي,"[فما, لبث, يوما, بسابق, مغنم, سرعتي, يوما, بسا...","[فما, لبث, يوما, بسابق, مغنم, سرعتي, يوما, بسا..."
2,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وتستوجبا منا علي وتحمدا,وإن تنظراني اليوم أقض لبانة,وان تنظراني اليوم اقض لبانه وتستوجبا منا علي و...,"[وان, تنظراني, اليوم, اقض, لبانه, وتستوجبا, من...","[وان, تنظراني, اليوم, اقض, لبانه, وتستوجبا, من..."
3,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,تؤامرني سرا لأصرم مرثدا,لعمرك ما نفس بجد رشيدة,لعمرك بجد رشيده تءامرني لاصرم مرثدا,"[لعمرك, بجد, رشيده, تءامرني, لاصرم, مرثدا]","[لعمرك, بجد, رشيده, تءامرني, لاصرم, مرثدا]"
4,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأفرع في لومي مرارا وأصعدا,وإن ظهرت منه قوارص جمة,وان ظهرت قوارص جمه وافرع لومي مرارا واصعدا,"[وان, ظهرت, قوارص, جمه, وافرع, لومي, مرارا, وا...","[وان, ظهرت, قوارص, جمه, وافرع, لومي, مرارا, وا..."
5,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,سوى قول باغ كادني فتجهدا,على غير ذنب أن أكون جنيته,علي ذنب ان اكون جنيته سوي قول باغ كادني فتجهدا,"[علي, ذنب, ان, اكون, جنيته, سوي, قول, باغ, كاد...","[علي, ذنب, ان, اكون, جنيته, سوي, قول, باغ, كاد..."
6,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,إذا ما المنادي في المقامة نددا,لعمري لنعم المرء تدعو بحبله,لعمري لنعم المرء تدعو بحبله اذا المنادي المقام...,"[لعمري, لنعم, المرء, تدعو, بحبله, اذا, المنادي...","[لعمري, لنعم, المرء, تدعو, بحبله, اذا, المنادي..."
7,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,ولا مؤيس منها إذا هو أوقدا,عظيم رماد القدر لا متعبس,عظيم رماد القدر متعبس مءيس اذا اوقدا,"[عظيم, رماد, القدر, متعبس, مءيس, اذا, اوقدا]","[عظيم, رماد, القدر, متعبس, مءيس, اذا, اوقدا]"
8,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,من الريح لم تترك لذي المال مرفدا,وإن صرحت كحل وهبت عرية,وان صرحت كحل وهبت عريه الريح تترك لذي المال مرفدا,"[وان, صرحت, كحل, وهبت, عريه, الريح, تترك, لذي,...","[وان, صرحت, كحل, وهبت, عريه, الريح, تترك, لذي,..."
9,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,إذا ضن ذو القربى عليهم وأخمدا,صبرت على وطء الموالي وحطمهم,صبرت علي وطء الموالي وحطمهم اذا ضن القربي عليه...,"[صبرت, علي, وطء, الموالي, وحطمهم, اذا, ضن, الق...","[صبرت, علي, وطء, الموالي, وحطمهم, اذا, ضن, الق..."


df.columns =['era', 'bard', 'boetical_works', 'rhyme','measure', 'l_hemistich', 'r_hemistich', 'line_of_poetry'] 

In [6]:
df.rename(columns={"العصر":"era"},inplace=True)
df.rename(columns={"الشاعر":"poet"},inplace=True)
df.rename(columns={"الديوان":"poetical_works"},inplace=True)
df.rename(columns={"القافية":"rhyme"},inplace=True)
df.rename(columns={"البحر":"measure"},inplace=True)
df.rename(columns={"البيت":"line_of_poetry"},inplace=True)
df.rename(columns={"الشطر الايمن":"l_hemistich"},inplace=True)
df.rename(columns={"الشطر الايسر":"r_hemistich"},inplace=True)

In [7]:
en = spacy.load('en_core_web_sm')
sw_spacy = en.Defaults.stop_words
print(sw_spacy)

C:\Users\mjeed\anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


{'above', 'about', 'now', 'besides', 'could', 'here', 'quite', 'himself', 'before', 'else', 'cannot', 'due', 'whose', 'whence', 'neither', 'towards', 'been', 'thereafter', 'seeming', '’d', 'move', 'few', 'my', 'two', 'used', 'whenever', 'throughout', 'has', 'empty', 'was', 'much', 'name', 'what', 'yet', 'ca', 'myself', 'on', 'when', 'whereupon', '‘s', 'top', 'however', 'whither', 'n’t', 'how', 'anyway', 'nowhere', 'very', 'so', 'though', 'either', 'rather', '‘ve', "'d", 'him', 'get', 'if', '‘re', 'eleven', 'toward', 'put', 'made', 'is', 'eight', 'unless', 'whatever', 'those', 'almost', 'always', 'why', 'nothing', 'us', 'amongst', 'four', 'from', 'nobody', 'some', 'ours', 'call', 'every', 'last', 'afterwards', 'any', 'mine', 'first', 'under', 'see', 'for', 'hundred', 'of', 'hereby', 'next', "'re", 'where', 'had', 're', 'ever', 'this', 'between', 'namely', 'whom', 'across', 'nor', 'ten', 'thereby', 'elsewhere', 'somewhere', 'your', 'indeed', 'no', 'does', 'sometime', 'nevertheless', 'as'

# تشكــــــــــــــــــــــيل

In [9]:
df["l_hemistich"] = df["l_hemistich"].astype(str)
df["r_hemistich"] = df["r_hemistich"].astype(str) 
df["poetical_works"] = df["poetical_works"].astype(str)
df["poet"] = df["poet"].astype(str)

In [10]:
def remove_tashkeel(text):
 #  Remove tashkeel
    text = text.replace("\u064B", "")  # ARABIC FATHATAN
    text = text.replace("\u064C", "")  # ARABIC DAMMATAN
    text = text.replace("\u064D", "")  # ARABIC KASRATAN
    text = text.replace("\u064E", "")  # ARABIC FATHA
    text = text.replace("\u064F", "")  # ARABIC DAMMA
    text = text.replace("\u0650", "")  # ARABIC KASRA
    text = text.replace("\u0651", "")  # ARABIC SHADDA
    text = text.replace("\u0652", "")  # ARABIC SUKUN
    text = text.replace("\u0653", "")  # ARABIC MADDAH ABOVE
    text = text.replace("\u0654", "")  # ARABIC HAMZA ABOVE
    text = text.replace("\u0655", "")  # ARABIC HAMZA BELOW
    text = text.replace("\u0656", "")  # ARABIC SUBSCRIPT ALEF
    text = text.replace("\u0657", "")  # ARABIC INVERTED DAMMA
    text = text.replace("\u0658", "")  # ARABIC MARK NOON GHUNNA
    text = text.replace("\u0659", "")  # ARABIC ZWARAKAY
    text = text.replace("\u065A", "")  # ARABIC VOWEL SIGN SMALL V ABOVE
    text = text.replace("\u065B", "")  # ARABIC VOWEL SIGN INVERTED SMALL V ABOVE
    text = text.replace("\u065C", "")  # ARABIC VOWEL SIGN DOT BELOW
    text = text.replace("\u065D", "")  # ARABIC REVERSED DAMMA
    text = text.replace("\u065E", "")  # ARABIC FATHA WITH TWO DOTS
    text = text.replace("\u065F", "")  # ARABIC WAVY HAMZA BELOW
    text = text.replace("\u0670", "")  # ARABIC LETTER SUPERSCRIPT ALEF

    return text


In [11]:
df['r_hemistich'] = df['r_hemistich'].apply(remove_tashkeel)
df['l_hemistich'] = df['l_hemistich'].apply(remove_tashkeel)
df['line_of_poetry'] = df['line_of_poetry'].apply(remove_tashkeel)
df['era'] = df['era'].apply(remove_tashkeel)
df['poet'] = df['poet'].apply(remove_tashkeel)
df['poetical_works'] = df['poetical_works'].apply(remove_tashkeel)

In [12]:
df

,era,poet,poetical_works,rhyme,measure,r_hemistich,l_hemistich,line_of_poetry
0,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأن تجمعا شملي وتنتظرا غدا,خليلي لا تستعجلا أن تزودا,خليلي لا تستعجلا أن تزودا وأن تجمعا شملي وت...
1,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,ولا سرعتي يوما بسابقة الردى,فما لبث يوما بسابق مغنم,فما لبث يوما بسابق مغنم ولا سرعتي يوما بساب...
2,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وتستوجبا منا علي وتحمدا,وإن تنظراني اليوم أقض لبانة,وإن تنظراني اليوم أقض لبانة وتستوجبا منا عل...
3,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,تؤامرني سرا لأصرم مرثدا,لعمرك ما نفس بجد رشيدة,لعمرك ما نفس بجد رشيدة تؤامرني سرا لأصرم مرثدا
4,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأفرع في لومي مرارا وأصعدا,وإن ظهرت منه قوارص جمة,وإن ظهرت منه قوارص جمة وأفرع في لومي مرارا ...
...,...,...,...,...,...,...,...,...
1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تتغنى,هي أغلى ما أنشأ الله في الدنيا,هي أغلى ما أنشأ الله في الدنيا وأحلى قصيدة ...
1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,هي أغرودة الأغاريد تنساب كحلم يغشى الجفون ا...
1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجدا ويخفق حسنا,هي شلال بهجة وبهاء,هي شلال بهجة وبهاء يتداعى وجدا ويخفق حسنا
1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجنا فسجنا,هي حلم الهوى ومنطلقي الباقي,هي حلم الهوى ومنطلقي الباقي يدك الحدود سجنا...


In [46]:
import nltk


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mjeed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:

nltk.download('stopwords')
stop_words = stopwords.words()
def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mjeed\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
df['line_of_poetry'] = df['line_of_poetry'].apply(preprocess)

In [52]:
# ازالة على باقي الكولم تاخذ وقت في الرن 
#df['r_hemistich'] = df['r_hemistich'].apply(preprocess)
#df['l_hemistich'] = df['l_hemistich'].apply(preprocess)
#df['era'] = df['era'].apply(preprocess)
#df['poet'] = df['poet'].apply(preprocess)
#df['poetical_works'] = df['poetical_works'].apply(preprocess)

In [53]:
df

,era,poet,poetical_works,rhyme,measure,r_hemistich,l_hemistich,line_of_poetry
0,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأن تجمعا شملي وتنتظرا غدا,خليلي لا تستعجلا أن تزودا,خليلي تستعجلا ان تزودا وان تجمعا شملي وتنتظرا
1,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,ولا سرعتي يوما بسابقة الردى,فما لبث يوما بسابق مغنم,فما لبث يوما بسابق مغنم سرعتي يوما بسابقه الردي
2,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وتستوجبا منا علي وتحمدا,وإن تنظراني اليوم أقض لبانة,وان تنظراني اليوم اقض لبانه وتستوجبا منا علي و...
3,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,تؤامرني سرا لأصرم مرثدا,لعمرك ما نفس بجد رشيدة,لعمرك بجد رشيده تءامرني لاصرم مرثدا
4,قبل الإسلام,عمرو بن قميئة,الديوان الرئيسي,د,الطويل,وأفرع في لومي مرارا وأصعدا,وإن ظهرت منه قوارص جمة,وان ظهرت قوارص جمه وافرع لومي مرارا واصعدا
...,...,...,...,...,...,...,...,...
1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تتغنى,هي أغلى ما أنشأ الله في الدنيا,اغلي انشا الله الدنيا واحلي قصيده تتغني
1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,اغروده الاغاريد تنساب كحلم يغشي الجفون الوسني
1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجدا ويخفق حسنا,هي شلال بهجة وبهاء,شلال بهجه وبهاء يتداعي وجدا ويخفق حسنا
1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجنا فسجنا,هي حلم الهوى ومنطلقي الباقي,حلم الهوي ومنطلقي الباقي يدك الحدود سجنا فسجنا


# تقسيـــــــــــــــــم

In [54]:
import spacy
nlp = spacy.load('en_core_web_sm')

C:\Users\mjeed\anaconda3\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [55]:
from nltk.tokenize import  word_tokenize

In [56]:
df["line_of_poetry_split"] = df["line_of_poetry"].apply(nltk.word_tokenize)

In [57]:
df["line_of_poetry_split"]

0          [خليلي, تستعجلا, ان, تزودا, وان, تجمعا, شملي, ...
1          [فما, لبث, يوما, بسابق, مغنم, سرعتي, يوما, بسا...
2          [وان, تنظراني, اليوم, اقض, لبانه, وتستوجبا, من...
3                 [لعمرك, بجد, رشيده, تءامرني, لاصرم, مرثدا]
4          [وان, ظهرت, قوارص, جمه, وافرع, لومي, مرارا, وا...
                                 ...                        
1831765      [اغلي, انشا, الله, الدنيا, واحلي, قصيده, تتغني]
1831766    [اغروده, الاغاريد, تنساب, كحلم, يغشي, الجفون, ...
1831767       [شلال, بهجه, وبهاء, يتداعي, وجدا, ويخفق, حسنا]
1831768    [حلم, الهوي, ومنطلقي, الباقي, يدك, الحدود, سجن...
1831769    [حبي, العاتي, وكل, غرامي, اه, ادرك, الغرام, لجنا]
Name: line_of_poetry_split, Length: 1831770, dtype: object

In [ ]:
# pos_tag

In [58]:
df["line_of_poetry_split"].apply(nltk.pos_tag)

0          [(خليلي, JJ), (تستعجلا, NNP), (ان, NNP), (تزود...
1          [(فما, JJ), (لبث, NNP), (يوما, NNP), (بسابق, N...
2          [(وان, JJ), (تنظراني, NNP), (اليوم, NNP), (اقض...
3          [(لعمرك, JJ), (بجد, NNP), (رشيده, NNP), (تءامر...
4          [(وان, JJ), (ظهرت, NNP), (قوارص, NNP), (جمه, N...
                                 ...                        
1831765    [(اغلي, JJ), (انشا, NNP), (الله, NNP), (الدنيا...
1831766    [(اغروده, JJ), (الاغاريد, NNP), (تنساب, NNP), ...
1831767    [(شلال, JJ), (بهجه, NNP), (وبهاء, NNP), (يتداع...
1831768    [(حلم, JJ), (الهوي, NNP), (ومنطلقي, NNP), (الب...
1831769    [(حبي, JJ), (العاتي, NNP), (وكل, NNP), (غرامي,...
Name: line_of_poetry_split, Length: 1831770, dtype: object

In [ ]:
df["line_of_poetry_split"]

In [ ]:
# stopwords

In [59]:
from nltk.corpus import stopwords
sw_nltk = stopwords.words('arabic')
print(sw_nltk)

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

In [61]:
sw_nltk = stopwords.words('arabic')
df["line_of_poetry_split_stop"]=df["line_of_poetry_split"].apply(lambda x :[item for item in x if item not in sw_nltk ])

In [63]:
df["line_of_poetry_split_stop"]

0          [خليلي, تستعجلا, ان, تزودا, وان, تجمعا, شملي, ...
1          [فما, لبث, يوما, بسابق, مغنم, سرعتي, يوما, بسا...
2          [وان, تنظراني, اليوم, اقض, لبانه, وتستوجبا, من...
3                 [لعمرك, بجد, رشيده, تءامرني, لاصرم, مرثدا]
4          [وان, ظهرت, قوارص, جمه, وافرع, لومي, مرارا, وا...
                                 ...                        
1831765      [اغلي, انشا, الله, الدنيا, واحلي, قصيده, تتغني]
1831766    [اغروده, الاغاريد, تنساب, كحلم, يغشي, الجفون, ...
1831767       [شلال, بهجه, وبهاء, يتداعي, وجدا, ويخفق, حسنا]
1831768    [حلم, الهوي, ومنطلقي, الباقي, يدك, الحدود, سجن...
1831769    [حبي, العاتي, وكل, غرامي, اه, ادرك, الغرام, لجنا]
Name: line_of_poetry_split_stop, Length: 1831770, dtype: object

In [70]:
x= df["line_of_poetry_split_stop"]

In [81]:
df.tail()

,era,poet,poetical_works,rhyme,measure,r_hemistich,l_hemistich,line_of_poetry,line_of_poetry_split,line_of_poetry_split_stop
1831765,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,وأحلى قصيدة تتغنى,هي أغلى ما أنشأ الله في الدنيا,اغلي انشا الله الدنيا واحلي قصيده تتغني,"[اغلي, انشا, الله, الدنيا, واحلي, قصيده, تتغني]","[اغلي, انشا, الله, الدنيا, واحلي, قصيده, تتغني]"
1831766,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,كحلم يغشى الجفون الوسنى,هي أغرودة الأغاريد تنساب,اغروده الاغاريد تنساب كحلم يغشي الجفون الوسني,"[اغروده, الاغاريد, تنساب, كحلم, يغشي, الجفون, ...","[اغروده, الاغاريد, تنساب, كحلم, يغشي, الجفون, ..."
1831767,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يتداعى وجدا ويخفق حسنا,هي شلال بهجة وبهاء,شلال بهجه وبهاء يتداعي وجدا ويخفق حسنا,"[شلال, بهجه, وبهاء, يتداعي, وجدا, ويخفق, حسنا]","[شلال, بهجه, وبهاء, يتداعي, وجدا, ويخفق, حسنا]"
1831768,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,يدك الحدود سجنا فسجنا,هي حلم الهوى ومنطلقي الباقي,حلم الهوي ومنطلقي الباقي يدك الحدود سجنا فسجنا,"[حلم, الهوي, ومنطلقي, الباقي, يدك, الحدود, سجن...","[حلم, الهوي, ومنطلقي, الباقي, يدك, الحدود, سجن..."
1831769,الحديث,شهاب غانم,شهاب غانم,ن,الخفيف,آه لو أدرك الغرام لجنا,هي حبي العاتي وكل غرامي,حبي العاتي وكل غرامي اه ادرك الغرام لجنا,"[حبي, العاتي, وكل, غرامي, اه, ادرك, الغرام, لجنا]","[حبي, العاتي, وكل, غرامي, اه, ادرك, الغرام, لجنا]"


In [94]:
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()

In [101]:
import json
import http.client

#####################
text = df["line_of_poetry_split_stop"]
#####################

text = text.replace('"', '')
conn = http.client.HTTPSConnection("farasa-api.qcri.org")
payload = "{\"text\":\"%s\"}"% text
payload = payload.encode('utf-8')

headers = { 'content-type': "application/json", 'cache-control': "no-cache", }

conn.request("POST", "/msa/webapi/lemma", payload, headers)

res = conn.getresponse()

data = res.read().decode('utf-8')
data_dict = json.loads(data)

TypeError: the JSON object must be str, bytes or bytearray, not DataFrame

In [92]:
df["line_of_poetry_split_ISRIStemmer"] = df["line_of_poetry_split"].apply(st.stem.decode('utf-8'))

AttributeError: 'function' object has no attribute 'decode'

In [98]:
from nltk.stem.isri import ISRIStemmer
st = ISRIStemmer()
w= df["line_of_poetry_split_stop"]
join = w.decode('Windows-1256')


AttributeError: 'Series' object has no attribute 'decode'

In [99]:
from nltk import word_tokenize
from nltk.stem.isri import ISRIStemmer

st = ISRIStemmer()
word_list = df["line_of_poetry_split_stop"]
# Define a function
def filter(word_list):
    wordsfilter=[]
    for a in word_tokenize(word_list):
        stem = st.stem(a)
        wordsfilter.append(stem)
    print(wordsfilter)

# Call the function
filter(word_list)

TypeError: expected string or bytes-like object